# Operations and Supply Chain Analytics - Assignment 1

**Due:** 5:00pm Eastern Time, March 31 (Sunday)  
**Submission:** Submit both a write-up and your Excel files electronically on Canvas.  
**Student:** Rodolfo Lerma

## Introduction

This Jupyter Notebook presents my solutions to the first assignment for the "Operations and Supply Chain Analytics" course. The assignment encompasses a range of problems designed to test our understanding and application of supply chain and operations analytics concepts, including office location optimization, production network planning, transport optimization, and inventory management. The solutions herein leverage Python and its libraries, such as PuLP for linear programming, to model and solve the given problems.

## Table of Contents

### Problem 1: Office Location and Consultant Assignment
  - In this problem, SC Consulting must decide on the optimal location of its home offices and the number of consultants to assign to each office, with the goal of minimizing costs. The problem considers annual fixed costs of locating offices, expected number of trips to each state, and travel costs from each potential site.

### Problem 2: Production Network for StayFresh
  - StayFresh, a refrigerator manufacturer in India, needs to plan its production network evolution over the next five years to accommodate anticipated demand growth. The problem involves deciding on capacity expansion at existing and potential plant sites, considering variable production and transport costs, as well as one-time capacity expansion costs.

### Problem 3: Optimal Engine Load
  - Harley-Davidson transports engines from its assembly plant in Milwaukee to its motorcycle assembly plant in Pennsylvania using trucks. The problem requires determining the optimal number of engines to load onto each truck, considering transportation costs, engine costs, and annual holding costs.

### Problem 4: JIT Manufacturing Impact
  - As part of a just-in-time (JIT) manufacturing initiative, Harley-Davidson has reduced the number of engines loaded on each truck. The problem asks to analyze the impact of this decision on annual inventory costs and determine the optimal truck cost for the given load of engines.

### Problem 5: Green Thumb Manufacturing Decision
  - Green Thumb, a manufacturer of lawn care equipment, has introduced a new product with a normally distributed demand. The problem involves determining the optimal number of units to manufacture, considering manufacturing costs, selling price, holding costs, and salvage value of unsold units. The goal is to maximize expected profit and estimate the average number of customers turned away due to stockouts.

## Problem 1: Office Location and Consultant Assignment

In [1]:
import pulp

# Initialize the problem
problem = pulp.LpProblem("SC_Consulting_Office_Location_Corrected", pulp.LpMinimize)

# Potential office locations with their fixed costs
locations = ["Los Angeles", "Tulsa", "Denver", "Seattle"]
fixed_costs = {"Los Angeles": 165428, "Tulsa": 131230, "Denver": 140000, "Seattle": 145000}

# Total number of trips required per state
trips_per_state = {
    "Washington": 40, "Oregon": 35, "California": 100, "Idaho": 25, "Nevada": 40,
    "Montana": 25, "Wyoming": 50, "Utah": 30, "Arizona": 50, "Colorado": 65,
    "New Mexico": 40, "North Dakota": 30, "South Dakota": 20, "Nebraska": 30,
    "Kansas": 40, "Oklahoma": 55
}

# Total number of trips needed
total_trips_needed = sum(trips_per_state.values())

# Cost dictionaries by location
cost_from_location = {
    "Los Angeles": {"Washington": 150, "Oregon": 150, "California": 75, "Idaho": 150, "Nevada": 100, "Montana": 175, "Wyoming": 150, "Utah": 150, "Arizona": 75, "Colorado": 150, "New Mexico": 125, "North Dakota": 300, "South Dakota": 300, "Nebraska": 250, "Kansas": 250, "Oklahoma": 250},
    "Tulsa": {"Washington": 250, "Oregon": 250, "California": 200, "Idaho": 200, "Nevada": 200, "Montana": 175, "Wyoming": 175, "Utah": 150, "Arizona": 200, "Colorado": 125, "New Mexico": 125, "North Dakota": 200, "South Dakota": 175, "Nebraska": 100, "Kansas": 75, "Oklahoma": 25},
    "Denver": {"Washington": 200, "Oregon": 200, "California": 150, "Idaho": 125, "Nevada": 125, "Montana": 125, "Wyoming": 100, "Utah": 100, "Arizona": 100, "Colorado": 25, "New Mexico": 75, "North Dakota": 150, "South Dakota": 125, "Nebraska": 125, "Kansas": 75, "Oklahoma": 125},
    "Seattle": {"Washington": 25, "Oregon": 75, "California": 125, "Idaho": 125, "Nevada": 150, "Montana": 125, "Wyoming": 150, "Utah": 200, "Arizona": 250, "Colorado": 250, "New Mexico": 300, "North Dakota": 200, "South Dakota": 200, "Nebraska": 250, "Kansas": 300, "Oklahoma": 300},
}

# Decision variable: number of consultants in each location
consultants = pulp.LpVariable.dicts("Consultants", locations, lowBound=0, cat='Continuous')

# Binary decision variables for opening an office in each location
office_open = pulp.LpVariable.dicts("OfficeOpen", locations, cat='Binary')

# Objective function: Modified to include fixed costs conditionally
total_cost = pulp.lpSum([
    fixed_costs[location] * office_open[location] +  # Only add fixed cost if office is open
    pulp.lpSum([cost_from_location[location][state] * trips_per_state[state] / 25 * consultants[location]
    for state in trips_per_state])  # Travel cost calculation
    for location in locations])

problem += total_cost

# Constraint: Ensure that the total number of consultant trips meets the requirement
problem += pulp.lpSum(consultants[location] * 25 for location in locations) >= sum(trips_per_state.values())

# Additional Constraints: Link the binary variable to the consultant variables
for location in locations:
    problem += consultants[location] <= office_open[location] * 1e5  # Big-M method to link the binary and continuous variables

# Solve the problem
problem.solve()

# Output results
if problem.status == pulp.LpStatusOptimal:
    print("Optimal Number of Consultants and Location:")
    for location in locations:
        if pulp.value(office_open[location]) == 1:  # Only display locations where an office is opened
            print(f"{location}: {pulp.value(consultants[location])} consultants")
    print(f"Total Annual Cost: ${pulp.value(problem.objective):,.2f}")
else:
    print("Problem could not be solved to optimality.")

Optimal Number of Consultants and Location:
Denver: 27.0 consultants
Total Annual Cost: $225,860.00


## Problem 2: Production Network for StayFresh

In this section, we address the StayFresh supply chain optimization problem, focusing on efficiently distributing refrigerators from manufacturing plants in Mumbai and Chennai to meet the regional demands across India. The challenge involves planning the production and potential expansion of facilities to accommodate a forecasted increase in demand while minimizing the associated costs.

**Objectives**

The primary objective is to minimize the total cost, which includes:
- The production and transportation costs from each plant to each regional market.
- The costs associated with expanding the production capacity at each plant location.

**Decision Variables**

- **Production Variables:** Represent the quantity of refrigerators produced at each plant for each regional market, across the five-year planning horizon.
- **Expansion Variables:** Indicate whether or not the company decides to expand the capacity at each plant location by either 150,000 or 300,000 units, in each year of the planning period.

**Constraints**

1. **Demand Satisfaction:** The total number of refrigerators produced and transported must meet or exceed the demand in each regional market, for each year.
2. **Capacity Limitation:** The production at each plant cannot exceed its available capacity, taking into account any expansions.

In [9]:
import pulp

# Define the problem
problem = pulp.LpProblem("StayFresh_Optimization", pulp.LpMinimize)

# Define the sets
plants = ["Chennai", "Delhi", "Kolkata", "Mumbai"]
regions = ["North", "East", "West", "South"]
years = list(range(1, 6))
capacity_options = [150, 300]

# Production and transport costs
costs = {
    "Chennai": {"North": 20, "East": 19, "West": 17, "South": 15},
    "Delhi": {"North": 15, "East": 18, "West": 17, "South": 20},
    "Kolkata": {"North": 18, "East": 15, "West": 20, "South": 19},
    "Mumbai": {"North": 17, "East": 20, "West": 15, "South": 17}
}

# Initial demand and growth rate
initial_demand = {"North": 100, "East": 50, "West": 150, "South": 150}
growth_rate = 0.20

# Expansion costs
expansion_costs = {150: 2000, 300: 3400}

# Current capacities
current_capacity = {"Chennai": 300, "Delhi": 0, "Kolkata": 0, "Mumbai": 300}

# Discount factor
discount_factor = 0.20

# Decision variables
production_vars = pulp.LpVariable.dicts("production", 
                                        (years, plants, regions),
                                        lowBound=0,
                                        cat='Continuous')

expansion_vars = pulp.LpVariable.dicts("expansion",
                                       (years, plants, capacity_options),
                                       cat='Binary')

# Objective function
objective = pulp.lpSum([
    production_vars[year][plant][region] * costs[plant][region] / (1 + discount_factor) ** year
    for year in years for plant in plants for region in regions
]) + pulp.lpSum([
    expansion_vars[year][plant][size] * expansion_costs[size] / (1 + discount_factor) ** year
    for year in years for plant in plants for size in capacity_options
])

problem += objective

# Constraints
# Demand constraints
for year in years:
    for region in regions:
        demand = initial_demand[region] * (1 + growth_rate) ** (year - 1)
        problem += pulp.lpSum([
            production_vars[year][plant][region] for plant in plants
        ]) >= demand, f"Demand_{region}_Year_{year}"

# Capacity constraints
for year in years:
    for plant in plants:
        problem += pulp.lpSum([
            production_vars[year][plant][region] for region in regions
        ]) <= current_capacity[plant] + pulp.lpSum([
            expansion_vars[y][plant][size] * size for y in range(1, year + 1) for size in capacity_options
        ]), f"Capacity_{plant}_Year_{year}"

# Solve the problem
problem.solve()

# Improved code for visualizing results with total production and detailed cost breakdown
print("Status:", pulp.LpStatus[problem.status])

for year in years:
    print(f"\nYear {year}")
    # Calculate total_demand for the current year considering growth_rate
    total_demand_year = sum(initial_demand[region] * (1 + growth_rate) ** (year - 1) for region in regions)
    print(f"Demand: {total_demand_year}")
    
    print("Production")
    total_yearly_production = 0
    for plant in plants:
        for region in regions:
            production_amount = pulp.value(production_vars[year][plant][region])
            if production_amount > 0:
                print(f"- Plant {plant}, Region {region}: {production_amount}")
                total_yearly_production += production_amount
    print(f"- TOTAL Production: {total_yearly_production}")

    print("Expansion (if any)")
    expansion_cost_for_year = 0
    for plant in plants:
        for size in capacity_options:
            if pulp.value(expansion_vars[year][plant][size]) > 0:
                expansion_type = f"150K" if size == 150 else "300K"
                print(f"- Plant {plant}, Expansion {expansion_type}: 1.0")
                expansion_cost_for_year += expansion_costs[size] * (pulp.value(expansion_vars[year][plant][size]))

    if expansion_cost_for_year == 0:
        print("- No Expansion")

    # Calculate and display the total cost for the year
    production_and_transportation_cost_for_year = sum(
        pulp.value(production_vars[year][plant][region]) * costs[plant][region]
        for plant in plants for region in regions if pulp.value(production_vars[year][plant][region]) > 0
    ) / 1000  # Convert to millions of Rupees for consistency
    total_cost_for_year = (production_and_transportation_cost_for_year + expansion_cost_for_year) / (1 + discount_factor) ** (year - 1)
    print(f"TOTAL Cost for Year {year}: {total_cost_for_year:.2f} Million Rupees")

# Calculate and print the total cost over the planning horizon correctly
total_cost_over_planning_horizon = sum(
    ((production_and_transportation_cost_for_year + expansion_cost_for_year) / (1 + discount_factor) ** (year - 1))
    for year in years
)
print(f"\nTotal Cost over the planning horizon: {total_cost_over_planning_horizon:.2f} Million Rupees")


Status: Optimal

Year 1
Demand: 450.0
Production
- Plant Chennai, Region East: 50.0
- Plant Chennai, Region South: 150.0
- Plant Mumbai, Region North: 100.0
- Plant Mumbai, Region West: 150.0
- TOTAL Production: 450.0
Expansion (if any)
- No Expansion
TOTAL Cost for Year 1: 7.15 Million Rupees

Year 2
Demand: 540.0
Production
- Plant Chennai, Region East: 60.0
- Plant Chennai, Region South: 180.0
- Plant Mumbai, Region North: 120.0
- Plant Mumbai, Region West: 180.0
- TOTAL Production: 540.0
Expansion (if any)
- No Expansion
TOTAL Cost for Year 2: 7.15 Million Rupees

Year 3
Demand: 648.0
Production
- Plant Chennai, Region South: 216.0
- Plant Kolkata, Region North: 60.0
- Plant Kolkata, Region East: 72.0
- Plant Mumbai, Region North: 84.0
- Plant Mumbai, Region West: 216.0
- TOTAL Production: 648.0
Expansion (if any)
- Plant Kolkata, Expansion 150K: 1.0
TOTAL Cost for Year 3: 1395.88 Million Rupees

Year 4
Demand: 777.5999999999999
Production
- Plant Chennai, Region South: 259.2
- Pla

In [5]:
import pulp

# Problem definition
prob = pulp.LpProblem("StayFresh_Optimized_Distribution", pulp.LpMinimize)

# Decision variables
locations = ["Chennai", "Delhi", "Kolkata", "Mumbai"]
regions = ["North", "East", "West", "South"]
years = range(1, 6)

production = pulp.LpVariable.dicts("Production", (locations, regions, years), lowBound=0, cat='Continuous')
expansion150 = pulp.LpVariable.dicts("Expansion150", (locations, years), cat='Binary')
expansion300 = pulp.LpVariable.dicts("Expansion300", (locations, years), cat='Binary')

# Constants
costs = {
    "Chennai": {"North": 20, "East": 19, "West": 17, "South": 15},
    "Delhi": {"North": 15, "East": 18, "West": 17, "South": 20},
    "Kolkata": {"North": 18, "East": 15, "West": 20, "South": 19},
    "Mumbai": {"North": 17, "East": 20, "West": 15, "South": 17}
}

initial_demand = {"North": 100, "East": 50, "West": 150, "South": 150}
growth_rate = 0.20
initial_capacity = {"Chennai": 300, "Delhi": 0, "Kolkata": 0, "Mumbai": 300}
expansion_costs = {150: 2000, 300: 3400}  # Costs are in thousands for consistency
discount_factor = 0.20

# Objective function
total_cost = (
    pulp.lpSum([
        production[loc][reg][year] * costs[loc][reg] * (1 / (1 + discount_factor) ** year)
        for loc in locations for reg in regions for year in years
    ])
    + pulp.lpSum([
        (expansion150[loc][year] * expansion_costs[150] + expansion300[loc][year] * expansion_costs[300]) * (1 / (1 + discount_factor) ** year)
        for loc in locations for year in years
    ])
)

prob += total_cost

# Constraints
for year in years:
    for reg in regions:
        prob += (
            pulp.lpSum([production[loc][reg][year] for loc in locations]) >= initial_demand[reg] * (1 + growth_rate) ** (year - 1),
            f"Demand_{reg}_Year_{year}"
        )

for year in years:
    for loc in locations:
        prob += (
            pulp.lpSum([production[loc][reg][year] for reg in regions]) <= initial_capacity[loc] + 150 * expansion150[loc][year] + 300 * expansion300[loc][year],
            f"Capacity_{loc}_Year_{year}"
        )

# Solve the problem
prob.solve()

# Results
print("Status:", pulp.LpStatus[prob.status])


Status: Optimal


In [6]:
# Results
print("Status:", pulp.LpStatus[prob.status])

for year in years:
    print(f"\nYear {year}")
    
    total_demand_year = sum(initial_demand[reg] * (1 + growth_rate) ** (year - 1) for reg in regions)
    print(f"Demand: {total_demand_year:.1f}")
    
    print("Production")
    total_production = 0
    for loc in locations:
        for reg in regions:
            prod_val = production[loc][reg][year].varValue  # Corrected indexing order to (loc, reg, year)
            if prod_val > 0:
                print(f"- Plant {loc}, Region {reg}: {prod_val:.1f}")
                total_production += prod_val
    print(f"- TOTAL Production: {total_production:.1f}")

    print("Expansion (if any)")
    expansions = []
    for loc in locations:
        if expansion150[loc][year].varValue > 0:
            expansions.append(f"- Plant {loc}, Expansion 150: {expansion150[loc][year].varValue}")
        if expansion300[loc][year].varValue > 0:
            expansions.append(f"- Plant {loc}, Expansion 300: {expansion300[loc][year].varValue}")
    
    if expansions:
        for expansion in expansions:
            print(expansion)
    else:
        print("- No Expansion")

    # Assuming the total cost calculation and presentation as required...
    # This part needs to calculate the total cost for the year based on production, transportation, and expansion costs.
    # Then it should display the calculated total cost for the year as specified.


Status: Optimal

Year 1
Demand: 450.0
Production
- Plant Chennai, Region East: 50.0
- Plant Chennai, Region South: 150.0
- Plant Mumbai, Region North: 100.0
- Plant Mumbai, Region West: 150.0
- TOTAL Production: 450.0
Expansion (if any)
- No Expansion

Year 2
Demand: 540.0
Production
- Plant Chennai, Region East: 60.0
- Plant Chennai, Region South: 180.0
- Plant Mumbai, Region North: 120.0
- Plant Mumbai, Region West: 180.0
- TOTAL Production: 540.0
Expansion (if any)
- No Expansion

Year 3
Demand: 648.0
Production
- Plant Chennai, Region East: 66.0
- Plant Chennai, Region South: 216.0
- Plant Delhi, Region North: 144.0
- Plant Delhi, Region East: 6.0
- Plant Mumbai, Region West: 216.0
- TOTAL Production: 648.0
Expansion (if any)
- Plant Delhi, Expansion 150: 1.0

Year 4
Demand: 777.6
Production
- Plant Chennai, Region South: 259.2
- Plant Delhi, Region North: 172.8
- Plant Delhi, Region East: 86.4
- Plant Mumbai, Region West: 259.2
- TOTAL Production: 777.6
Expansion (if any)
- Plant 

In [7]:
import pulp

def optimize_distribution(growth_rate):
    # Problem definition
    prob = pulp.LpProblem("StayFresh_Optimized_Distribution", pulp.LpMinimize)

    # Decision variables
    locations = ["Chennai", "Delhi", "Kolkata", "Mumbai"]
    regions = ["North", "East", "West", "South"]
    years = range(1, 6)

    production = pulp.LpVariable.dicts("Production", (years, locations, regions), lowBound=0, cat='Continuous')
    expansion150 = pulp.LpVariable.dicts("Expansion150", (locations, years), cat='Binary')
    expansion300 = pulp.LpVariable.dicts("Expansion300", (locations, years), cat='Binary')

    # Constants
    costs = {
        "Chennai": {"North": 20, "East": 19, "West": 17, "South": 15},
        "Delhi": {"North": 15, "East": 18, "West": 17, "South": 20},
        "Kolkata": {"North": 18, "East": 15, "West": 20, "South": 19},
        "Mumbai": {"North": 17, "East": 20, "West": 15, "South": 17}
    }

    initial_demand = {"North": 100, "East": 50, "West": 150, "South": 150}
    initial_capacity = {"Chennai": 300, "Delhi": 0, "Kolkata": 0, "Mumbai": 300}
    expansion_costs = {150: 2000, 300: 3400}  # Costs are in thousands for consistency
    discount_factor = 0.20

    # Objective function
    total_cost = (
        pulp.lpSum([
            production[year][loc][reg] * costs[loc][reg] * (1 / (1 + discount_factor) ** year)
            for loc in locations for reg in regions for year in years
        ])
        + pulp.lpSum([
            (expansion150[loc][year] * expansion_costs[150] + expansion300[loc][year] * expansion_costs[300]) * (1 / (1 + discount_factor) ** year)
            for loc in locations for year in years
        ])
    )

    prob += total_cost

    # Constraints
    for year in years:
        for reg in regions:
            prob += (
                pulp.lpSum([production[year][loc][reg] for loc in locations]) >= initial_demand[reg] * (1 + growth_rate) ** (year - 1),
                f"Demand_{reg}_Year_{year}"
            )

    for year in years:
        for loc in locations:
            prob += (
                pulp.lpSum([production[year][loc][reg] for reg in regions]) <= initial_capacity[loc] + 150 * expansion150[loc][year] + 300 * expansion300[loc][year],
                f"Capacity_{loc}_Year_{year}"
            )

    # Solve the problem
    prob.solve()

    # Results for each year, with the specified format
    for year in years:
        print(f"\nYear {year}")
        demand_year = sum(initial_demand[reg] * (1 + growth_rate) ** (year - 1) for reg in regions)
        print(f"Demand: {demand_year:.1f}")
        
        print("Production")
        total_production_year = 0
        for loc in locations:
            for reg in regions:
                prod_amount = pulp.value(production[year][loc][reg])
                if prod_amount > 0:
                    print(f"- Plant {loc}, Region {reg}: {prod_amount}")
                    total_production_year += prod_amount
        print(f"- TOTAL Production: {total_production_year}")
        
        expansion_flag = False
        for loc in locations:
            if any(pulp.value(expansion150[loc][year]) > 0 for size in [150, 300]):
                expansion_flag = True
                print(f"- Plant {loc}, Expansion 150: {pulp.value(expansion150[loc][year])}")
            if any(pulp.value(expansion300[loc][year]) > 0 for size in [150, 300]):
                expansion_flag = True
                print(f"- Plant {loc}, Expansion 300: {pulp.value(expansion300[loc][year])}")
                
        if not expansion_flag:
            print("Expansion (if any)")
            print("- No Expansion")

# Running the optimization for different growth rates
for growth_rate in [0.15, 0.25]:
    print(f"Optimizing distribution with an anticipated growth rate of {growth_rate * 100}%\n")
    optimize_distribution(growth_rate)

# Call the function with different growth rates
for growth_rate in [0.15, 0.25]:
    optimize_distribution(growth_rate)


Optimizing distribution with an anticipated growth rate of 15.0%


Year 1
Demand: 450.0
Production
- Plant Chennai, Region East: 50.0
- Plant Chennai, Region South: 150.0
- Plant Mumbai, Region North: 100.0
- Plant Mumbai, Region West: 150.0
- TOTAL Production: 450.0
Expansion (if any)
- No Expansion

Year 2
Demand: 517.5
Production
- Plant Chennai, Region East: 57.5
- Plant Chennai, Region South: 172.5
- Plant Mumbai, Region North: 115.0
- Plant Mumbai, Region West: 172.5
- TOTAL Production: 517.5
Expansion (if any)
- No Expansion

Year 3
Demand: 595.1
Production
- Plant Chennai, Region East: 66.125
- Plant Chennai, Region West: 30.625
- Plant Chennai, Region South: 198.375
- Plant Mumbai, Region North: 132.25
- Plant Mumbai, Region West: 167.75
- TOTAL Production: 595.125
Expansion (if any)
- No Expansion

Year 4
Demand: 684.4
Production
- Plant Chennai, Region East: 71.86875
- Plant Chennai, Region South: 228.13125
- Plant Delhi, Region North: 145.825
- Plant Delhi, Region East: 4.1

## Problem 3: Optimal Engine Load

In [ ]:
# Given data
cost_per_trip = 1000
holding_cost_rate = 0.20
engine_cost = 500
daily_demand = 300

# Calculate optimal load size based on the Economic Order Quantity formula
EOQ = ((2 * daily_demand * 365 * cost_per_trip) / (holding_cost_rate * engine_cost)) ** 0.5

print(f"Optimal number of engines per truck: {EOQ}")

## Problem 4: JIT Manufacturing Impact

In [ ]:
# Given data
cost_per_trip = 1000
engine_load = 100
holding_cost_rate = 0.20
engine_cost = 500

# Calculate annual inventory cost under JIT
annual_inventory_cost = (engine_load / 2) * engine_cost * holding_cost_rate

print(f"Annual inventory cost with JIT: {annual_inventory_cost}")

## Problem 5: Green Thumb Manufacturing Decision

In [ ]:
from scipy.stats import norm

# Given data
cost_per_unit = 150
price_per_unit = 200
salvage_value = 50
holding_cost = 20
mean_demand = 100
std_demand = 40

# Calculate the optimal order quantity based on the Newsvendor model

# Critical ratio
critical_ratio = (price_per_unit - cost_per_unit) / (price_per_unit - salvage_value)

# Find the Z-score corresponding to the critical ratio
Z = norm.ppf(critical_ratio)

# Optimal order quantity
optimal_order_quantity = mean_demand + Z * std_demand

print(f"Optimal units to manufacture: {optimal_order_quantity}")